In [1]:
import numpy as np
import pandas as pd
import healpy as hp
from joblib import Parallel, delayed
from tqdm import tqdm
import os
from pylab import cm
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.colors
from matplotlib.colors import Normalize
from matplotlib.colorbar import ColorbarBase

nside = 8
npix = hp.nside2npix(nside)
# Create directory for intermediate output
file_inter = "all_samples"
os.makedirs(file_inter, exist_ok=True)

df = pd.read_csv('masked_sample.dat', sep="\t", header=None)
df.columns = ['r', 'th', 'ph','z']

####################################################################################################

d = 0.4436
for i in range(10):
    fd=df[(df['z']>=(0.36+i*d/2)) &(df['z']<=(0.36+i*d/2+d))]
    # fd.drop(['z'], axis=1, inplace=True)
    fd_new = fd.drop(['z'], axis=1)

    f_in = f'all_samples/sample_{i+1}.dat'
    fd_new.to_csv(f_in,sep="\t",header= None, index=False)    # save each samples
    
    th = fd_new['th'].to_numpy()
    ph = fd_new['ph'].to_numpy()
    r = fd_new['r'].to_numpy()

    npix = hp.nside2npix(nside)  # 12 * nside^2
    m1 = np.zeros(npix)
    
    for j in range(len(th)):
        px=hp.ang2pix(nside,th[j],ph[j])
        m1[px] += 1

###################################################
    pix = []
    for j in range(npix):
        if(m1[j]!=0):
            pix.append(j)

    pix = np.array(pix)
    df_pix = pd.DataFrame(data=pix)
    f_pix = f'all_samples/non_zero_pix_id_s{i+1}.dat'
    df_pix.to_csv(f_pix, sep='\t', header=None, index=False)
# #################################################### 
os.remove('masked_sample.dat')